In [ ]:
import pandas as pd

df = pd.read_csv('./flight_data_processed.csv')

print(df.tail())


In [ ]:
df['Routine'] = df['Departure Airport'] + '-' + df['Arrival Airport']

df = df.drop(columns=['Flight Number', 'Duration', 'Departure Airport', 'Arrival Airport', 'Arrival Time'])

df.head()

In [ ]:
import numpy as np

num_parts = 50
df_parts = np.array_split(df, num_parts)


In [7]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

mydb = mysql.connector.connect(
  host=os.getenv('DB_CLOUD_HOST'),
  user=os.getenv('DB_CLOUD_USER'),
  password=os.getenv('DB_CLOUD_PASSWORD'),
  database=os.getenv('DB_CLOUD_NAME'),
  ssl_disabled=True,
  connection_timeout=1000000,
)

mycursor = mydb.cursor()

print("Đã kết nối thành công đến MySQL!")



Đã kết nối thành công đến MySQL!


In [ ]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS flights (
    id INT AUTO_INCREMENT PRIMARY KEY,
    departure_time DATETIME,
    carrier_code VARCHAR(5),
    price DECIMAL(10, 2),
    routine VARCHAR(20)
)
"""

try:
    mycursor.execute(create_table_sql)
    print("Đang khởi tạo bảng flights")

    insert_sql = """
    INSERT INTO flights 
    (departure_time, carrier_code, price, routine) 
    VALUES (%s, %s, %s, %s)
    """

    total_inserted = 0
    for i, df_part in enumerate(df_parts):
        values = df_part[['Departure Time', 'Carrier Code', 'Price (VND)', 'Routine']].values.tolist()
        
        mycursor.executemany(insert_sql, values)
        mydb.commit()
        
        total_inserted += len(values)
        print(f"Đã chèn thành công {len(values)} bản ghi từ phần {i+1}/{len(df_parts)}. Tổng: {total_inserted}")

    print(f"Đã chèn thành công tổng cộng {total_inserted} bản ghi vào bảng flights.")
except mysql.connector.Error as error:
    print(f"Lỗi khi thao tác với MySQL: {error}")

In [6]:
mycursor.execute("SELECT * FROM flights")

rows = mycursor.fetchall()

print("Dữ liệu từ bảng flights:")
for row in rows:
    print(row)


Dữ liệu từ bảng flights:
(1, datetime.datetime(2024, 9, 15, 5, 25), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(2, datetime.datetime(2024, 9, 15, 6, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(3, datetime.datetime(2024, 9, 15, 6, 40), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(4, datetime.datetime(2024, 9, 15, 7, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(5, datetime.datetime(2024, 9, 15, 8, 10), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(6, datetime.datetime(2024, 9, 15, 8, 45), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(7, datetime.datetime(2024, 9, 15, 10, 10), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(8, datetime.datetime(2024, 9, 15, 11, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(9, datetime.datetime(2024, 9, 15, 11, 25), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(10, datetime.datetime(2024, 9, 15, 12, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(11, datetime.datetime(2024, 9, 15, 12, 30), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(12, datetime.datetime(2024, 9, 15, 13, 20), 'VJ', Decimal(

In [ ]:
if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("Kết nối MySQL đã đóng.")